In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip,BiSkip

dir_st = 'data/skip-thoughts'
vocab = ['robots', 'are', 'very', 'cool', '<eos>', 'BiDiBu']
biskip = BiSkip(dir_st, vocab)
uniskip = UniSkip(dir_st, vocab)

input = Variable(torch.LongTensor([
    [1,2,3,4,0], # robots are very cool 0
    [5,2,3,0,0]  # bidibu are very cool <eos>
])) # <eos> token is optional

print(input)
def generate_embeding(batch):
    top_half = uniskip(batch)
    bottom_half = biskip(batch)
    combine_skip = torch.cat([top_half,bottom_half],dim=1) 
    return combine_skip

print(generate_embeding(input))
print(generate_embeding(input).size())

/usr/cs/undergrad/2019/gnosborn/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[1, 2, 3, 4, 0],
        [5, 2, 3, 0, 0]])
tensor([[ 0.1793, -0.1286,  0.0964,  ..., -0.3654, -0.0070, -0.0919],
        [ 0.1372, -0.1739, -0.0526,  ..., -0.1531,  0.0139, -0.1812]],
       grad_fn=<CatBackward>)
torch.Size([2, 4800])


In [2]:
import pandas as pd
from Vocabulary import Vocabulary, preprocess

In [84]:
def make_vocab(tokens):
    voc = Vocabulary(['<PAD>','<UNK>'])
    voc.add_tokens(tokens)
    print('vocab len is {}'.format(len(voc.w2idx)))
    return voc

In [85]:
def load_data(file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv'):
    '''TODO remove 10% of data for hyper param tuning'''
    df= pd.read_csv(file)
    df = df.drop('InputStoryid',axis=1)
    targets = df['AnswerRightEnding']
    df = df.drop('AnswerRightEnding',axis=1)
    df = df.drop('InputSentence1',axis=1)
    df = df.drop('InputSentence2',axis=1)
    df = df.drop('InputSentence3',axis=1)
    
    voc_str= ''
    for index, row in df.iterrows():
        voc_str+=' '.join(list(row)) + ' '
        
    df['AnswerRightEnding'] = targets
    return df, make_vocab(preprocess(voc_str))

In [86]:
   
df, voc = load_data()


vocab len is 5226


In [87]:
df.head()

,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,I danced terribly and broke a friend's toe.,My friends decided to keep inviting me out as ...,"The next weekend, I was asked to please stay h...",2
1,My eyes were watery and it was hard to breathe.,My allergies were too bad and I had to go back...,It reminded me of how much I loved spring flow...,1
2,She made poor decisions that night and was unf...,Avery thought her children would be happy with...,Avery regretted what she did the next day.,2
3,The eggs his mom used must have been bad though.,Josh thought that the pie was delicious.,Josh got sick.,2
4,"He took a walk, hung out with some friends, an...",He felt inspiration and then went back home to...,John then got an idea for his painting.,1


In [88]:
df['RandomFifthSentenceQuiz1'][0],

('My friends decided to keep inviting me out as I am so much fun.',)

In [89]:
voc.get_sentence(preprocess(df['RandomFifthSentenceQuiz1'][0]))  

[11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23]

In [90]:
import sys
class LastSentenceDataset(Dataset):
    '''currently implements no context model. will add in last sentence later'''
    def __init__(self,file='story_cloze_data/cloze_test_test__spring2016 - cloze_test_ALL_test.csv',vocab=None):

        super().__init__()
        df, created_vocab = load_data(file)
        
        if vocab:
            self.vocab = vocab
        else:
            self.vocab = created_vocab
        #self.df = df
      
        
        self.dir_st = 'data/skip-thoughts'
        self.biskip = BiSkip(self.dir_st, self.vocab.convert_to_list())
        
        self.uniskip = UniSkip(self.dir_st, self.vocab.convert_to_list())
        
       
        self.data = self.make_data(df)
        
        
    def __getitem__(self, idx):
        """
        Args:
            idx
        Returns: skip thought embedding of ending and 0/1 if it is the right ending 

        """
        return self.data[idx]

    def __len__(self):
        """
        Returns len of the dataset
        """
        return len(self.data)
       
    def make_data(self, df):
        data = []
        total = df.index[:1000]
        print('skip thought encoding dataset')
        for i in total:
            #print(row['RandomFifthSentenceQuiz1'],row['RandomFifthSentenceQuiz2'])
           
            progress(i,len(total))
            endings =  self.gen_embbeding(df.at[i,'RandomFifthSentenceQuiz1'], df.at[i,'RandomFifthSentenceQuiz2'])
            if df.at[i,'AnswerRightEnding'] == 1:
                data.append((endings[0],1))
                data.append((endings[1],0))
            else:
                data.append((endings[0],0))
                data.append((endings[1],1))
        return data
    

    def zero_pad(self,l,n):
        l = (l + n * [0])[:n]
        return l
    
    def pad_input(self,a,b):
        ed = sorted([a,b],key=len)
        longer = ed[1]
        shorter = ed[0]
        padded = self.zero_pad( shorter,len(longer))
        if shorter == a:
            return padded,b
        else: return a,padded
        
    def gen_embbeding(self,sent1,sent2):
        sent1 = preprocess(sent1)
        sent2 = preprocess(sent2)
        #remove random n token that is in one sentence
        if 'n' in sent2:
            sent2.remove('n')
        encoded_end1 = self.vocab.get_sentence(sent1)
        encoded_end2 = self.vocab.get_sentence(sent2)
        a,b = self.pad_input(encoded_end1,encoded_end2)
        
        batch = torch.LongTensor([a,b]) 
        top_half = self.uniskip(batch)
        bottom_half = self.biskip(batch)
        combine_skip = torch.cat([top_half,bottom_half],dim=1) 
        return combine_skip    
    
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [91]:
class NoContextModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = torch.nn.Linear(4800,256)
        self.hidden= torch.nn.Linear(256,64)
        self.output = torch.nn.Linear(64,2)
    def forward(self, inputs):
        hidden = torch.nn.functional.relu(self.input(inputs))
        hidden1 = torch.nn.functional.relu(self.hidden(hidden))
        output = self.output(hidden1)
        return output

In [12]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hd1 = torch.nn.Linear(4800, 2400)
        self.hd2 = torch.nn.Linear(2400, 1200)
        self.hd3 = torch.nn.Linear(1200, 600)
        self.output = torch.nn.Linear(600, 2)

    def forward(self, x):
        x = torch.nn.functional.relu(self.hd1(x))
        x = torch.nn.functional.relu(self.hd2(x))
        x = torch.nn.functional.relu(self.hd3(x))
        x = self.output(x)
        #print('output',x)
        return x

In [25]:
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('seaborn-paper')
batch_size = 64
lr = 0.01
num_epochs = 5
report_every = 1


In [103]:
dataset = LastSentenceDataset()
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False,num_workers = 0)

vocab len is 5226


/usr/cs/undergrad/2019/gnosborn/nlp/env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


skip thought encoding dataset


In [26]:
model = NoContextModel()
if torch.cuda.is_available():
    model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)

In [27]:
tick = time.time()
epoch_losses = []
for epoch_num in range(1, num_epochs + 1):
    batch_losses = []
    for i, batch in enumerate(data_loader):
        ### YOUR CODE BELOW ###
        # Zero the gradients
        #print('Zero Grad')
        optimizer.zero_grad()
        # Extract the inputs and the targets
        #print('Extract')
        inputs, targets = batch
        #print(inputs.shape)
        #print(inputs)
        # Transfer the inputs and the targets to GPUs, if available
        #print('2cuda start')
        if torch.cuda.is_available():
            #print("transfering to Cuda")
            inputs = torch.FloatTensor(inputs.float()).cuda()
            targets = torch.LongTensor(targets).cuda()
        #print('2cuda end')
        #print(inputs.size())
        # Run the model
        #print('model')
        outputs = model(inputs)

        # Compute the loss
        #print('loss')
        loss = criterion(outputs,targets)
        
        
        # Backpropagate the error
        #print('backprop')
        loss.backward(retain_graph=True)
        # Update the parameters
        #print('step')
        optimizer.step()
        
        # Append the loss
        #print('append loss')
        batch_losses.append(float (loss))
        ### YOUR CODE ABOVE ###
        epoch_loss = np.mean(np.array(batch_losses))
        epoch_losses.append(epoch_loss)

        if epoch_num % report_every == 0:
            tock = time.time()
            print("Epoch {}. Loss {:.4f}. Elapsed {:.0f} seconds".format(epoch_num, epoch_loss, tock-tick))

print("Total time elapsed: {:.0f} minutes".format((tock-tick)/60))

Epoch 1. Loss 0.6920. Elapsed 26 seconds
Epoch 1. Loss 0.8636. Elapsed 52 seconds
Epoch 1. Loss 2.1122. Elapsed 77 seconds
Epoch 1. Loss 1.7683. Elapsed 102 seconds
Epoch 1. Loss 1.5619. Elapsed 126 seconds
Epoch 1. Loss 1.4208. Elapsed 149 seconds
Epoch 1. Loss 1.3193. Elapsed 173 seconds
Epoch 1. Loss 1.2443. Elapsed 198 seconds
Epoch 1. Loss 1.1813. Elapsed 222 seconds
Epoch 1. Loss 1.1337. Elapsed 246 seconds
Epoch 1. Loss 1.0929. Elapsed 269 seconds
Epoch 1. Loss 1.0586. Elapsed 293 seconds
Epoch 1. Loss 1.0299. Elapsed 317 seconds
Epoch 1. Loss 1.0065. Elapsed 342 seconds
Epoch 1. Loss 0.9864. Elapsed 367 seconds
Epoch 1. Loss 0.9686. Elapsed 391 seconds
Epoch 1. Loss 0.9536. Elapsed 417 seconds
Epoch 1. Loss 0.9380. Elapsed 442 seconds
Epoch 1. Loss 0.9260. Elapsed 466 seconds
Epoch 1. Loss 0.9138. Elapsed 491 seconds
Epoch 1. Loss 0.9038. Elapsed 515 seconds
Epoch 1. Loss 0.8935. Elapsed 540 seconds
Epoch 1. Loss 0.8852. Elapsed 566 seconds
Epoch 1. Loss 0.8769. Elapsed 589 sec

Epoch 7. Loss 0.2619. Elapsed 4604 seconds
Epoch 7. Loss 0.2598. Elapsed 4630 seconds
Epoch 7. Loss 0.2367. Elapsed 4655 seconds
Epoch 7. Loss 0.2232. Elapsed 4679 seconds
Epoch 7. Loss 0.1988. Elapsed 4703 seconds
Epoch 7. Loss 0.1917. Elapsed 4728 seconds
Epoch 7. Loss 0.1883. Elapsed 4754 seconds
Epoch 7. Loss 0.1840. Elapsed 4779 seconds
Epoch 7. Loss 0.1774. Elapsed 4803 seconds
Epoch 7. Loss 0.1788. Elapsed 4828 seconds
Epoch 7. Loss 0.1815. Elapsed 4853 seconds
Epoch 7. Loss 0.1803. Elapsed 4878 seconds
Epoch 7. Loss 0.1759. Elapsed 4902 seconds
Epoch 7. Loss 0.1740. Elapsed 4927 seconds
Epoch 7. Loss 0.1682. Elapsed 4952 seconds
Epoch 7. Loss 0.1624. Elapsed 4976 seconds
Epoch 7. Loss 0.1609. Elapsed 5000 seconds
Epoch 7. Loss 0.1589. Elapsed 5025 seconds
Epoch 7. Loss 0.1567. Elapsed 5049 seconds
Epoch 7. Loss 0.1537. Elapsed 5073 seconds
Epoch 7. Loss 0.1515. Elapsed 5098 seconds
Epoch 7. Loss 0.1507. Elapsed 5123 seconds
Epoch 7. Loss 0.1508. Elapsed 5145 seconds
Epoch 7. Lo

In [ ]:
next(model.parameters()).is_cuda

In [33]:
def save(name):
    torch.save(model.state_dict(), 'saved_models/{}.save'.format(name))


In [31]:
def load(name):
    l_model = NoContextModel()
    l_model.load_state_dict(torch.load('saved_models/{}.save'.format(name)))

In [112]:
def score(pair):
    '''true if model predicts right'''
    inpt, target = pair
    softm = torch.nn.Softmax(dim=0)
    dist = softm(model(inpt.cuda()))
    pred = dist.argmax()
    #print(dist,target)
    return pred.item() == target


In [104]:
vocab = dataset.vocab
val_data_set = LastSentenceDataset(file='story_cloze_data/cloze_test_val__spring2016 - cloze_test_ALL_val.csv',vocab=vocab)

vocab len is 5303
skip thought encoding dataset


In [117]:
num_right = 0
for i in val_data_set:
    if score(i):
        num_right+=1
num_right/len(val_data_set)

0.575